In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models,Input
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


In [ ]:
# Load dataset
data_train = np.load('./drive/MyDrive/train_dataset3.npz')
# data_test = np.load(test_dir)
# X = data_train['X']
# X_test = data_test['X']
# y = data_train['y']
# y_test = data_test['y']
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']
class_names=data_train['class_names']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data_train['X'], data_train['y'], test_size=0.2, random_state=42)

print(f"Loaded dataset:")
# X_train = X_train
# X_test = X_test / 255.0

print(f"Loaded dataset after rescaling:")
print(f"X_train shape: {X_train.shape}, max: {X_train.max()}, min: {X_train.min()}")
print(f"X_test shape: {X_test.shape}, max: {X_test.max()}, min: {X_test.min()}")


In [ ]:
def plot_sample(X, y, index):
    plt.figure(figsize=(3, 2))
    plt.imshow(X[index] )  # Multiply back by 255 for visualization
    plt.xlabel(class_names[y[index]])



In [ ]:
# Example usage of plot_sample
plot_sample(X_train, y_train, 5)

In [ ]:

cnn = models.Sequential([
    Input(shape=(64, 64, 3)),
    # layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    # layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.GlobalAveragePooling2D(),  # Replaces Flatten
    layers.Dense(64, activation='relu'),
    layers.Dense(14, activation='softmax')
])


learning_rate = 0.0005  # Example: Use a smaller learning rate for fine-tuning
optimizer = Adam(learning_rate=learning_rate)

cnn.compile(optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])


In [ ]:

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize
    return image, label

# Create tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

# Train with the dataset
cnn.fit(train_dataset, epochs=50)
cnn.evaluate(test_dataset)


In [ ]:
# cnn.fit(X_train,y_train,epochs=50,batch_size=64)

In [ ]:
model_h5_path = "./drive/MyDrive/FYP/cnn_model.h5"
cnn.save(model_h5_path)  # Save in HDF5 format
print(f"\nModel saved in HDF5 format at: {model_h5_path}")

In [ ]:
# Plot accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
cnn.evaluate(X_test,y_test)

In [ ]:
y_pred= cnn.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]
print("classification_report\n",classification_report(y_test,y_pred_classes))
y_pred_classes[:5]

In [ ]:
y_test[:5]


In [ ]:
plot_sample(X_test,y_test,4)

In [ ]:
class_names[y_pred_classes[4]]